In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import subprocess
import sys

from xml.dom.minidom import parse, parseString

import numpy as np

import pandas as pd
import tifffile
from skimage.transform import rescale

import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

from phenix import fiji
from phenix import correction_image  
import phenix


In [3]:
pwd

'/n/core/micro/jeg/vah/081222_inducedamage/RPE1_inducedamage_100822__2022-10-11T16_35_29-Measurement 1/PhenixStitching'

In [4]:
'''
Enter the path to the images folder and the folder to save tiles to.

image_path : str
    the path to the phenix Images folder.

pxml_file : str
    the path to the xml file inside the images folder. This should always be named Index.idx.xml
    
savedir : str
    the name of the folder where projected tiles will be saved 
'''

image_path = "../Images"
pxml_file = f"{image_path}/Index.idx.xml"

savedir = "../concatenated_2"

In [5]:
'''
Read the xml file and create a dataframe.
Just run this cell, nothing need to be set.

xmldata : xml
    xml object from the phenix xml file

df : DataFrame
    Dataframe with image information (field, position, channel, width, ....)
    
bdf : DataFrame
    DataFrame with positions of each tile
'''

xmldata = parse(pxml_file)
df = phenix.dataframe_from_xml(xmldata)
bdf = df.groupby(["well", "field"]).max(numeric_only=True).reset_index()

In [6]:
welldf = df.groupby("well").max(numeric_only=True).reset_index()

In [8]:
'''
Create a plot of tile positions. Select the the tiles to stitch.
'''
output_notebook()

ds_dict = {'x_values':bdf.posx, 'y_values':bdf.posy}
ds = ColumnDataSource(data=welldf)

def app(doc):
    bfig = figure(width=800, height=800, tools=["box_select", "reset", "wheel_zoom", "box_zoom"],
                  match_aspect=True)
    bfig.circle(x='col', y='row', source=ds, size=30, alpha=.25)
    bfig.text(x='col', y='row', text='well', source=ds,
             text_align="center", text_baseline="middle",
             text_font_size={'value':'20px'})
    
    bfig.y_range.flipped = True
    doc.add_root(bfig)
    
show(app, notebook_url='http://localhost:8888')

Loading BokehJS ...

In [6]:
'''
Create a dataframe that contains the tiles selected about
Nothing need to be set, just run the cell.
'''

topdf, fielddf = phenix.crop_to_rect_bokeh(ds, bdf, df)

4 4


In [9]:
topdf = df
fielddf = bdf

In [10]:
'''
Just run this cell.
Setting the correction image to have the correct number of channels
'''
nchannels = int(topdf.channel.max())
if nchannels < len(correction_image):
    xcor = correction_image[:nchannels]
else:
    xcor = correction_image

In [14]:
'''
Run this to max/sum/None project tiles and save into savedir.

'''

savedir = "../sum_2"
projection = "SUM"
#savedir = "b2_09_20"

if not os.path.exists(savedir):
    os.mkdir(savedir)

fg = topdf.groupby(["field", 'well'])

tiles = list()

colors = ["green", "magenta", "yellow", "blue"]
for f, gdf in fg:
    tilefile = phenix.field_stack(gdf, image_path, xcor, savedir,
                                  projection, f[1], f[0], colors=colors)
    
    print(f)
#     tilex = gdf.xoffset.mean()
#     tiley = gdf.yoffset.mean()
#     tilez = 0
#     tiles.append(f"{tilefile}; ; ({tilex:8.1f}, {tiley:8.1f}, {tilez:8.1f})")
    #print(tiles[-1])
    
# with open(f"{savedir}/gc_tile_config.txt", 'w') as tf:
#     tf.write("dim = 3\n")
#     tf.write("\n".join(tiles))

(1, 'b01')
(1, 'b02')
(1, 'b03')
(1, 'b04')
(1, 'b05')
(2, 'b01')
(2, 'b02')
(2, 'b03')
(2, 'b04')
(2, 'b05')
(3, 'b01')
(3, 'b02')
(3, 'b03')
(3, 'b04')
(3, 'b05')
(4, 'b01')
(4, 'b02')
(4, 'b03')
(4, 'b04')
(4, 'b05')
